In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
sess = tf.InteractiveSession()

In [4]:
def Weight_variables(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) #stddev是标准差, 返回充斥截断正态分布值的tensor,shape是输入
    return tf.Variable(initial) #定义这个tensor为variable

def bias_variables(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [5]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') #W其实是kernel.也其实是权重,名字没那么重要了.

def max_pool_2x2(x):
    #返回一个Tensor，类型不变，shape仍然是[batch, height, width, channels]这种形式
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
# input x , y_ for true labels
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None,10])

x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1是代表样本数量不变, 1代表1个颜色通道.


In [7]:

#定义卷积
# 第一层

W_conv1 = Weight_variables([5,5,1,32]) # 5x5 的kernel, 1个颜色通道(gray),其实输入也就是1层, 32个卷积核,对应的会提取32个特征
b_conv1 = bias_variables([32]) # 对应32个特征
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # wx+b ,  relu ,  
h_pool1 = max_pool_2x2(h_conv1)


#### 第二层的输入
[5,5,32,64] 的 32 是第一层的输出的特征点

In [8]:
# 定义第二层
W_conv2 = Weight_variables([5,5,32,64]) # 32个输入特征,64个输出特征
b_conv2 = bias_variables([64])
h_conv2 = tf.nn.relu((conv2d(h_pool1,W_conv2) + b_conv2))
h_pool2 = max_pool_2x2(h_conv2)

#### 全连接层
两次max pooling之后,deci了两次, 28x28 --> 7x7了.
第二层输出64个单位.

In [9]:
# 定义全连接层
W_fc1 = Weight_variables([7*7*64, 1024]) #隐含节点是1024个
b_fc1 = bias_variables([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) #把第二层给平坦化
h_fc1 = tf.nn.relu( tf.matmul(h_pool2_flat, W_fc1) + b_fc1)



In [10]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [11]:
#经sigmoid得到概率表达
W_fc2 = Weight_variables([1024, 10])
b_fc2 = bias_variables([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [12]:
#损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(cross_entropy)


In [13]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [17]:
tf.global_variables_initializer().run()
for i in range(80000):
    batch =  mnist.train.next_batch(50)
    if i%100 == 0:
        train_acc = acc.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step, train acc", i, train_acc)
    
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

step, train acc 0 0.04
step, train acc 100 0.1
step, train acc 200 0.06
step, train acc 300 0.14
step, train acc 400 0.14
step, train acc 500 0.18
step, train acc 600 0.18
step, train acc 700 0.1
step, train acc 800 0.14
step, train acc 900 0.1
step, train acc 1000 0.14
step, train acc 1100 0.14
step, train acc 1200 0.04
step, train acc 1300 0.1
step, train acc 1400 0.08
step, train acc 1500 0.06
step, train acc 1600 0.04
step, train acc 1700 0.16
step, train acc 1800 0.1
step, train acc 1900 0.18
step, train acc 2000 0.14
step, train acc 2100 0.16
step, train acc 2200 0.1
step, train acc 2300 0.08
step, train acc 2400 0.28
step, train acc 2500 0.22
step, train acc 2600 0.14
step, train acc 2700 0.1
step, train acc 2800 0.2
step, train acc 2900 0.1
step, train acc 3000 0.22
step, train acc 3100 0.16
step, train acc 3200 0.16
step, train acc 3300 0.2
step, train acc 3400 0.06
step, train acc 3500 0.16
step, train acc 3600 0.18
step, train acc 3700 0.12
step, train acc 3800 0.16
step, tr

step, train acc 31100 0.4
step, train acc 31200 0.46
step, train acc 31300 0.4
step, train acc 31400 0.42
step, train acc 31500 0.42
step, train acc 31600 0.42
step, train acc 31700 0.34
step, train acc 31800 0.36
step, train acc 31900 0.34
step, train acc 32000 0.4
step, train acc 32100 0.44
step, train acc 32200 0.36
step, train acc 32300 0.32
step, train acc 32400 0.3
step, train acc 32500 0.48
step, train acc 32600 0.44
step, train acc 32700 0.36
step, train acc 32800 0.32
step, train acc 32900 0.32
step, train acc 33000 0.3
step, train acc 33100 0.44
step, train acc 33200 0.32
step, train acc 33300 0.34
step, train acc 33400 0.36
step, train acc 33500 0.48
step, train acc 33600 0.36
step, train acc 33700 0.4
step, train acc 33800 0.44
step, train acc 33900 0.48
step, train acc 34000 0.4
step, train acc 34100 0.34
step, train acc 34200 0.38
step, train acc 34300 0.36
step, train acc 34400 0.38
step, train acc 34500 0.32
step, train acc 34600 0.46
step, train acc 34700 0.36
step, tr

step, train acc 61700 0.66
step, train acc 61800 0.62
step, train acc 61900 0.44
step, train acc 62000 0.54
step, train acc 62100 0.44
step, train acc 62200 0.44
step, train acc 62300 0.64
step, train acc 62400 0.54
step, train acc 62500 0.58
step, train acc 62600 0.64
step, train acc 62700 0.54
step, train acc 62800 0.5
step, train acc 62900 0.54
step, train acc 63000 0.64
step, train acc 63100 0.62
step, train acc 63200 0.52
step, train acc 63300 0.58
step, train acc 63400 0.52
step, train acc 63500 0.54
step, train acc 63600 0.46
step, train acc 63700 0.56
step, train acc 63800 0.56
step, train acc 63900 0.56
step, train acc 64000 0.54
step, train acc 64100 0.52
step, train acc 64200 0.48
step, train acc 64300 0.6
step, train acc 64400 0.44
step, train acc 64500 0.56
step, train acc 64600 0.56
step, train acc 64700 0.66
step, train acc 64800 0.56
step, train acc 64900 0.6
step, train acc 65000 0.7
step, train acc 65100 0.52
step, train acc 65200 0.6
step, train acc 65300 0.64
step, 

In [18]:
acc.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})

0.6474

#### cpu和gpu速度
cpu会比较慢
gpu速度较快.